In [6]:
import pygame
import random
import math

# Inicialización de pygame
pygame.init()

# Dimensiones de la pantalla
screen_width = 800
screen_height = 600
screen = pygame.display.set_mode((screen_width, screen_height))

# Título e ícono
pygame.display.set_caption("Space Invaders")
icon = pygame.image.load('icon.jpg')  # Cambia este icono según tu imagen
pygame.display.set_icon(icon)

# Cargar imágenes de la nave y los invasores
player_img = pygame.image.load('player.jpg')  # Cambia según tu imagen
enemy_img = pygame.image.load('enemy.jpg')  # Cambia según tu imagen
bullet_img = pygame.image.load('bullet.jpg')  # Cambia según tu imagen

# Posición inicial de la nave
player_x = 370
player_y = 480
player_x_change = 0

# Invasores
num_of_enemies = 6
enemy_x = []
enemy_y = []
enemy_x_change = []
enemy_y_change = []
enemy_speed = 0.3

for i in range(num_of_enemies):
    enemy_x.append(random.randint(0, screen_width - 64))
    enemy_y.append(random.randint(50, 150))
    enemy_x_change.append(enemy_speed)
    enemy_y_change.append(40)

# Bala
bullet_x = 0
bullet_y = player_y
bullet_y_change = 1
bullet_state = "ready"  # Estado 'ready' significa que la bala no se ve en pantalla

# Puntuación
score_value = 0
font = pygame.font.Font('freesansbold.ttf', 32)
text_x = 10
text_y = 10

# Fin del juego
game_over_font = pygame.font.Font('freesansbold.ttf', 64)

def show_score(x, y):
    score = font.render(f"Puntuación: {score_value}", True, (255, 255, 255))
    screen.blit(score, (x, y))

def game_over_text():
    over_text = game_over_font.render("GAME OVER", True, (255, 255, 255))
    screen.blit(over_text, (200, 250))

def player(x, y):
    screen.blit(player_img, (x, y))

def enemy(x, y, i):
    screen.blit(enemy_img, (x, y))

def fire_bullet(x, y):
    global bullet_state
    bullet_state = "fire"
    screen.blit(bullet_img, (x + 16, y + 10))

def is_collision(enemy_x, enemy_y, bullet_x, bullet_y):
    distance = math.sqrt(math.pow(enemy_x - bullet_x, 2) + math.pow(enemy_y - bullet_y, 2))
    return distance < 27

# Ciclo principal del juego
running = True
while running:
    screen.fill((0, 0, 0))  # Color de fondo
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        # Movimiento de la nave
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_LEFT:
                player_x_change = -0.5
            if event.key == pygame.K_RIGHT:
                player_x_change = 0.5
            if event.key == pygame.K_SPACE:
                if bullet_state == "ready":
                    bullet_x = player_x
                    fire_bullet(bullet_x, bullet_y)

        if event.type == pygame.KEYUP:
            if event.key == pygame.K_LEFT or event.key == pygame.K_RIGHT:
                player_x_change = 0

    # Movimiento de la nave
    player_x += player_x_change
    player_x = max(0, min(player_x, screen_width - 64))

    # Movimiento del enemigo
    for i in range(num_of_enemies):
        # Fin del juego
        if enemy_y[i] > 440:
            for j in range(num_of_enemies):
                enemy_y[j] = 2000
            game_over_text()
            break

        enemy_x[i] += enemy_x_change[i]
        if enemy_x[i] <= 0 or enemy_x[i] >= screen_width - 64:
            enemy_x_change[i] *= -1
            enemy_y[i] += enemy_y_change[i]

        # Colisión
        collision = is_collision(enemy_x[i], enemy_y[i], bullet_x, bullet_y)
        if collision:
            bullet_y = player_y
            bullet_state = "ready"
            score_value += 10
            enemy_x[i] = random.randint(0, screen_width - 64)
            enemy_y[i] = random.randint(50, 150)

        enemy(enemy_x[i], enemy_y[i], i)

    # Movimiento de la bala
    if bullet_y <= 0:
        bullet_y = player_y
        bullet_state = "ready"

    if bullet_state == "fire":
        fire_bullet(bullet_x, bullet_y)
        bullet_y -= bullet_y_change

    # Aumenta la velocidad de los enemigos a medida que aumenta la puntuación
    if score_value % 50 == 0 and score_value != 0:
        for k in range(num_of_enemies):
            enemy_x_change[k] *= 1.05

    player(player_x, player_y)
    show_score(text_x, text_y)
    pygame.display.update()

pygame.quit()